In [1]:
import numpy as np
import pandas as pd

# Data load

In [10]:
df = pd.read_csv('study.csv')
df.head()

,individual,Ng10.a,Ng10.b,Ng18.a,Ng18.b,Ng8.a,Ng8.b,Bd12.a,Bd12.b,Ng20.a,...,stem_width,stem_height,first_flower_day,fruits_number,fruit_length,fruit_diameter,fruit_weight,seeds_number,yield,variety
0,1,1,1,4,4,2,2,2,2,1,...,1.52,78.0,44.0,13,23.18,3.39,48.08,135.00,624.98,cayenne
1,2,1,1,4,4,2,2,2,2,1,...,1.36,74.0,45.0,9,25.05,3.08,48.10,164.00,432.90,cayenne
2,3,1,1,3,3,2,2,2,2,1,...,1.43,69.0,63.0,8,17.75,2.03,19.05,47.00,152.40,cayenne
3,4,1,1,4,4,2,2,2,2,1,...,1.53,68.0,44.0,12,21.50,3.04,46.58,156.00,558.90,cayenne
4,5,1,1,4,4,2,2,2,2,1,...,1.34,84.0,44.0,6,24.88,3.38,54.05,148.33,324.30,cayenne


# Data split by type

## Genotypes

In [11]:
gen = df.iloc[:, 0:15].astype(str)
gen.head()

,individual,Ng10.a,Ng10.b,Ng18.a,Ng18.b,Ng8.a,Ng8.b,Bd12.a,Bd12.b,Ng20.a,Ng20.b,Ng6.a,Ng6.b,Ng7.a,Ng7.b
0,1,1,1,4,4,2,2,2,2,1,1,2,2,0,0
1,2,1,1,4,4,2,2,2,2,1,1,2,2,0,0
2,3,1,1,3,3,2,2,2,2,1,1,2,2,2,2
3,4,1,1,4,4,2,2,2,2,1,1,2,2,2,2
4,5,1,1,4,4,2,2,2,2,1,1,2,2,2,2


In [12]:
gen.dtypes

individual    object
Ng10.a        object
Ng10.b        object
Ng18.a        object
Ng18.b        object
Ng8.a         object
Ng8.b         object
Bd12.a        object
Bd12.b        object
Ng20.a        object
Ng20.b        object
Ng6.a         object
Ng6.b         object
Ng7.a         object
Ng7.b         object
dtype: object

## Morphological traits

In [14]:
mor = pd.concat([df.iloc[:, [0]], df.iloc[:, 15:-1]], axis=1)
mor.head()

,individual,stem_width,stem_height,first_flower_day,fruits_number,fruit_length,fruit_diameter,fruit_weight,seeds_number,yield
0,1,1.52,78.0,44.0,13,23.18,3.39,48.08,135.00,624.98
1,2,1.36,74.0,45.0,9,25.05,3.08,48.10,164.00,432.90
2,3,1.43,69.0,63.0,8,17.75,2.03,19.05,47.00,152.40
3,4,1.53,68.0,44.0,12,21.50,3.04,46.58,156.00,558.90
4,5,1.34,84.0,44.0,6,24.88,3.38,54.05,148.33,324.30


In [15]:
gen3 = gen.iloc[:3, :]
gen3.head()

,individual,Ng10.a,Ng10.b,Ng18.a,Ng18.b,Ng8.a,Ng8.b,Bd12.a,Bd12.b,Ng20.a,Ng20.b,Ng6.a,Ng6.b,Ng7.a,Ng7.b
0,1,1,1,4,4,2,2,2,2,1,1,2,2,0,0
1,2,1,1,4,4,2,2,2,2,1,1,2,2,0,0
2,3,1,1,3,3,2,2,2,2,1,1,2,2,2,2


In [16]:
df = gen
bases = sorted({col[:-2] for col in df.columns if col.endswith(('.a', '.b'))})

new_df = pd.DataFrame()

for base in bases:
    col_a = base + ".a"
    col_b = base + ".b"
    
    # combine values into a comma-separated string
    new_df[base] = df[col_a].astype(str) + "," + df[col_b].astype(str)

print(new_df)

    Bd12 Ng10 Ng18 Ng20  Ng6  Ng7  Ng8
0    2,2  1,1  4,4  1,1  2,2  0,0  2,2
1    2,2  1,1  4,4  1,1  2,2  0,0  2,2
2    2,2  1,1  3,3  1,1  2,2  2,2  2,2
3    2,2  1,1  4,4  1,1  2,2  2,2  2,2
4    2,2  1,1  4,4  1,1  2,2  2,2  2,2
..   ...  ...  ...  ...  ...  ...  ...
109  2,2  2,2  2,2  1,1  1,1  3,3  1,1
110  0,0  2,2  2,2  1,1  1,1  3,3  1,1
111  1,1  2,2  2,2  1,1  1,1  3,3  1,1
112  1,1  2,2  2,2  1,1  1,1  3,3  1,1
113  2,2  0,0  2,2  1,1  1,1  3,3  1,1

[114 rows x 7 columns]


In [17]:
import pandas as pd
import itertools

def pairwise_row_matches(df):
    results = []
    row_pairs = list(itertools.combinations(range(len(df)), 2))

    for i, j in row_pairs:
        row_i = df.iloc[i].astype(str)
        row_j = df.iloc[j].astype(str)

        total_matches = 0

        for col in df.columns:
            # Split by comma → ['2','2'], ['2','3'], etc.
            elems_i = row_i[col].split(",")
            elems_j = row_j[col].split(",")

            # Count element-wise matches (pairwise position)
            matches = sum(a == b for a, b in zip(elems_i, elems_j))
            total_matches += matches

        results.append({
            "Comparison": f"{i+1}_{j+1}",
            "Matches": total_matches
        })
    
    return pd.DataFrame(results)

In [19]:
import pandas as pd
import itertools
from typing import Iterable

def pairwise_relatedness(df: pd.DataFrame, decimals: int = 4) -> pd.DataFrame:
    """
    For every pair of rows in df, compute relatedness = matches / total_possible_elements.
    - Cells are split by comma and stripped of surrounding whitespace.
    - Matches are counted position-wise; missing positions count as non-matches.
    - total_possible_elements per column = max(len(elems_row_i), len(elems_row_j))
    Returns a DataFrame with columns: Comparison, relatedness (float between 0 and 1).
    """
    def split_cell(cell: object) -> list:
        # handle NaN, None, numeric values, etc.
        if pd.isna(cell):
            return []
        s = str(cell)
        # if empty string after stripping → no elements
        s = s.strip()
        if s == "":
            return []
        # split on comma and strip spaces from each element
        return [elem.strip() for elem in s.split(",")]

    results = []
    nrows = len(df)
    row_pairs = list(itertools.combinations(range(nrows), 2))

    # Pre-split all cells for speed
    split_cache = {
        r: {col: split_cell(df.iloc[r][col]) for col in df.columns}
        for r in range(nrows)
    }

    for i, j in row_pairs:
        total_matches = 0
        total_possible = 0

        for col in df.columns:
            elems_i = split_cache[i][col]
            elems_j = split_cache[j][col]

            max_len = max(len(elems_i), len(elems_j))
            total_possible += max_len

            # Compare position-wise: pad shorter with None
            for k in range(max_len):
                a = elems_i[k] if k < len(elems_i) else None
                b = elems_j[k] if k < len(elems_j) else None
                if a is not None and b is not None and a == b:
                    total_matches += 1
                # else: no match (including cases where one is None)

        # avoid division by zero (if total_possible==0, set relatedness to 0)
        relatedness = (total_matches / total_possible) if total_possible > 0 else 0.0
        results.append({
            "Comparison": f"{i+1}_{j+1}",
            "relatedness": round(relatedness, decimals)
        })

    return pd.DataFrame(results)

In [20]:
pairwise_relatedness(df)

,Comparison,relatedness
0,1_2,0.9333
1,1_3,0.6667
2,1_4,0.8000
3,1_5,0.8000
4,1_6,0.6667
...,...,...
6436,111_113,0.8000
6437,111_114,0.6667
6438,112_113,0.9333
6439,112_114,0.6667


In [16]:
plt(df.relatedness)

NameError: name 'plt' is not defined